# DNBLab Jupyter Notebook Tutorial

## SRU - Schnittstellenabfrage, Datenauslieferung und Volltextanalyse

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage zu digitalisierten Inhaltsverzeichnissen über die SRU-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst eine exemplarische Abfrage, Speichern der Inhaltsverzeichnisse aös Textdateien und Durchsuchen der Volltexte nach einem Stichwort. 

## Einrichten der Arbeitsumgebung <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die SRU-Schnittstelle wird BeautifulSoup https://www.crummy.com/software/BeautifulSoup/ und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit Pandas https://pandas.pydata.org/ können Elemente aus dem MARC21-Format ausgelesen werden.

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## SRU-Abfrage mit Ausgabe in MARC21-xml<a class="anchor" id="Teil2"></a>

Die Funktion dnb_sru nimmt den Paramter "query" der SRU-Abfrage entgegen und liefert alle Ergebnisse als eine Liste von Records aus. Bei mehr als 100 Records werden weitere Datensätze mit "&startRecord=101" abgerufen (mögliche Werte 1 bis 99.000). Weitere Informationen und Funktionen der SRU- Schnittstelle werden unter https://www.dnb.de/sru beschrieben.

In [2]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

### Durchsuchen eines MARC-Feldes<a class="anchor" id="Teil3"></a>

Die Funktion parse_records nimmt als Parameter jeweils ein Record entgegen und sucht über xpath die gewünschte Informationen heraus und liefert diese als Dictionary zurück. Die Schlüssel-Werte-Paare können beliebig agepasst und erweitert werden. In diesem Fall werden nur die Permalinks zu den digitalisierten Inhaltsverzeichnissen ausgegeben.

In [3]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #link
    link = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        link = link[0].text
    except:
        link = "unkown"
        
    meta_dict = {"link":link + '/text'}
    
    return meta_dict

Über die verschiedenen Indices https://services.dnb.de/sru/dnb?operation=explain&version=1.1 kann die SRU-Abfrage mittels CQL gezielt z.B. über das Suchhwort "Sandwespe" im Volltextindex der digitalisierten Inhaltsverzeichnisse eingeschränkt werden. Auf diese Art kann durch Anpassen des Suchwortes nach verschiedenen Begriffen in in den Inhaltsverzeichnissen gesucht werden. 

In [4]:
records = dnb_sru('inh=Sandwespe')
print(len(records), 'Ergebnisse')

35 Ergebnisse


## Beispielanzeige zur weiteren Bearbeitung <a class="anchor" id="Teil4"></a>

Mit der "Python Data Analysis Library" Pandas für Python wird das Ergebnis (Dictionary-Element "link") als Dataframe ausgegeben.

In [5]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,link
0,https://d-nb.info/1216718180/04/text
1,https://d-nb.info/1205215212/04/text
2,https://d-nb.info/1203042655/04/text
3,https://d-nb.info/1197947922/04/text
4,https://d-nb.info/1155773403/04/text
5,https://d-nb.info/1162289392/04/text
6,https://d-nb.info/113822071X/04/text
7,https://d-nb.info/1028151314/04/text
8,https://d-nb.info/1048291391/04/text
9,https://d-nb.info/1010103253/04/text


Verschiedene Funktionen zur Ausgabe der ermittelten Links:

In [6]:
#print(df.to_string(index=False))
#HTML(df.to_html(index=False))
#document = df.to_dict(orient='list')
#print(document)

Mit der folgenden Funktion df.to_csv() werden die Ergebnisse als links.csv in das Jupyter-Verzeichnins der Einstiegsseite abgelegt und können dort heruntergeladen werden. 

In [7]:
df.to_csv("links.csv", index=False)

Download der in der CSV-Datei gespeicherten Links zu den Textdateien im temporären Jupyter-Verzeichnis als text, text.1, text.2, usw.

In [8]:
!wget -i links.csv

--2021-07-22 05:46:23--  http://link/
Resolving link (link)... failed: No address associated with hostname.
wget: unable to resolve host address ‘link’
--2021-07-22 05:46:23--  https://d-nb.info/1216718180/04/text
Resolving d-nb.info (d-nb.info)... 193.175.100.223
Connecting to d-nb.info (d-nb.info)|193.175.100.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5192 (5.1K) [text/plain]
Saving to: ‘text’

text                100%[===================>]   5.07K  --.-KB/s    in 0s      

2021-07-22 05:46:25 (101 MB/s) - ‘text’ saved [5192/5192]

--2021-07-22 05:46:25--  https://d-nb.info/1205215212/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 1553 (1.5K) [text/plain]
Saving to: ‘text.1’

text.1              100%[===================>]   1.52K  --.-KB/s    in 0s      

2021-07-22 05:46:25 (107 MB/s) - ‘text.1’ saved [1553/1553]

--2021-07-22 05:46:25--  https://d-nb.info/1203042655/04/text
Reusing exis

HTTP request sent, awaiting response... 200 OK
Length: 8161 (8.0K) [text/plain]
Saving to: ‘text.23’

text.23             100%[===================>]   7.97K  --.-KB/s    in 0s      

2021-07-22 05:46:27 (153 MB/s) - ‘text.23’ saved [8161/8161]

--2021-07-22 05:46:27--  https://d-nb.info/451709578/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 29718 (29K) [text/plain]
Saving to: ‘text.24’

text.24             100%[===================>]  29.02K  --.-KB/s    in 0s      

2021-07-22 05:46:27 (82.5 MB/s) - ‘text.24’ saved [29718/29718]

--2021-07-22 05:46:27--  https://d-nb.info/368150585/04/text
Reusing existing connection to d-nb.info:443.
HTTP request sent, awaiting response... 200 OK
Length: 4910 (4.8K) [text/plain]
Saving to: ‘text.25’

text.25             100%[===================>]   4.79K  --.-KB/s    in 0s      

2021-07-22 05:46:27 (107 MB/s) - ‘text.25’ saved [4910/4910]

--2021-07-22 05:46:27--  https://d-nb.info/36700

Die heruntergeladenen Textdateien der Inhaltsverzeichnisse können nach einem Suchwort, z.B. search = "Wild" durchsucht. Hierbei wird die Groß- und Kleinschreibung beachtet und die Ergebnisse entsprechend der Zeilen in den einzelnen Dateien ausgegben. Das Suchwort kann beliebig geändert werden.
Die Treffer werden mit Angabe der Zeile und Datei ausgegeben. Dabei entspricht die Dateibenennung den im Verzeichnis heruntergeladen Textdateien (text, text1, text2 usw.).

In [9]:
search = 'biene'

filename = 'text'
with open(filename) as f:
    for num, line in enumerate(f, 1):
        if search in line:
            print('%s - found at line in text:' % search, num)
filename2 = 'text.1'
with open(filename2) as f:
    for num, line in enumerate(f, 1):
        if search in line:
            print('%s - found at line in text.1:' % search, num)

biene - found at line in text: 11
biene - found at line in text: 13
biene - found at line in text: 14
biene - found at line in text: 15
biene - found at line in text: 16
biene - found at line in text: 17
biene - found at line in text: 18
biene - found at line in text: 31
biene - found at line in text: 33
biene - found at line in text: 43
biene - found at line in text.1: 43
